In [51]:
import json
from functools import lru_cache
from operator import itemgetter
from typing import List
import boto3
import pandas as pd
import requests
from datetime import datetime
from pydantic import BaseModel, confloat, AnyUrl,conint
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL
import awswrangler as wr
import cryptocompare

In [52]:
crypto_compare_key = "6188db038cad874de5eb7da3821490d45967ae07911bb365904d2b1759400ea4"
cryptocompare.cryptocompare._set_api_key_parameter(crypto_compare_key)

'&api_key=6188db038cad874de5eb7da3821490d45967ae07911bb365904d2b1759400ea4'

In [53]:
session = boto3.Session(profile_name='master')
dynamo = session.resource('dynamodb')
s3_client = session.client('s3')

In [54]:
list_of_coins = [
    "BTC",
    "ETH",
    "BUSD",
    "USDT",
    "XRP",
    "SOL",
    "BNB",
    "DOT",
    "SHIB",
    "LTC"
  ]
list_of_currencies = ['USD']

In [55]:
df = wr.s3.read_csv('s3://kosmobiker-masterproject/data/coin_list/table/', boto3_session=session)


In [58]:
tmp_df = df[df['Name'].isin(list_of_coins)][['Name', 'ContentCreatedOn']]
created_on = dict(zip(tmp_df.Name,tmp_df.ContentCreatedOn))
created_on

{'BTC': 1417635237,
 'DOT': 1593791455,
 'USDT': 1499420073,
 'BUSD': 1569227610,
 'BNB': 1501700700,
 'SHIB': 1618843684,
 'SOL': 1586353077,
 'XRP': 1428671962,
 'LTC': 1426004323,
 'ETH': 1438259812}

In [98]:
def get_historical_data(coin:str,
                        cur:str,
                        created_on:dict,
                        ts=int(datetime.now().timestamp()),
                        limit=2000):
    data = []
    done = False
    while not done:
        try:
            tmp_json = cryptocompare.get_historical_price_hour(coin, cur, limit=limit, toTs=ts)
            if ts < created_on[coin]:
                done = True
            else:
                data.extend(tmp_json)
                ts -= 3600000
                done = False
        except Exception as err:
            print(err)
            done = True
    df = pd.DataFrame(data)
    df['coin'] = coin
    df['currency'] = cur
    return df 

In [143]:
df = get_historical_data('ETH', "USD", created_on)

In [147]:
df['date_time'] = pd.to_datetime(df['time'], unit='s', errors='raise')
df['year'] = df['date_time'].dt.year
df['month'] = df['date_time'].dt.month
df['day'] = df['date_time'].dt.day
df['hour'] = df['date_time'].dt.hour
df['minute'] = df['date_time'].dt.minute
df['delta'] = (df['close'] - df['open'])*100/df['open']
df.rename(mapper={
        'time' : 'time_stamp',
        'volumefrom' : 'volume_fsym',
        'volumeto' : 'volume_tsym',
        'coin' : 'ticker'
            }, axis=1, inplace=True)

df

,time_stamp,high,low,open,volume_fsym,volume_tsym,close,conversionType,conversionSymbol,coin_currency,date_time,year,month,day,hour,minute,delta
0,1653199200,1973.510,1969.280,1970.100,1650.43,3253977.56,1972.450,direct,,ETH_USD,2022-05-22 06:00:00,2022,5,22,6,0,0.119283
1,1653202800,1974.600,1968.280,1972.450,2329.18,4592315.11,1973.320,direct,,ETH_USD,2022-05-22 07:00:00,2022,5,22,7,0,0.044108
2,1653206400,1983.220,1971.020,1973.320,2852.21,5639338.82,1981.820,direct,,ETH_USD,2022-05-22 08:00:00,2022,5,22,8,0,0.430746
3,1653210000,2030.460,1981.820,1981.820,13354.09,26804068.70,2026.460,direct,,ETH_USD,2022-05-22 09:00:00,2022,5,22,9,0,2.252475
4,1653213600,2034.570,2023.560,2026.460,6404.42,12993160.61,2025.670,direct,,ETH_USD,2022-05-22 10:00:00,2022,5,22,10,0,-0.038984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124057,1440784800,1.210,1.201,1.205,21410.72,25759.57,1.203,multiply,BTC,ETH_USD,2015-08-28 18:00:00,2015,8,28,18,0,-0.165975
124058,1440788400,1.199,1.189,1.203,16399.47,19540.44,1.192,multiply,BTC,ETH_USD,2015-08-28 19:00:00,2015,8,28,19,0,-0.914381
124059,1440792000,1.204,1.198,1.192,15173.67,18234.33,1.202,multiply,BTC,ETH_USD,2015-08-28 20:00:00,2015,8,28,20,0,0.838926
124060,1440795600,1.205,1.201,1.202,9948.66,11973.10,1.203,multiply,BTC,ETH_USD,2015-08-28 21:00:00,2015,8,28,21,0,0.083195


In [150]:
ttt = ['ticker', 'date_time', 'open', 'high', 'low', 'close', 'volume_fsym', 'volume_tsym',
                     'currency', 'delta', 'time_stamp', 'year', 'month', 'day', 'hour', 'minute']

for tt in ttt:
    print(f"""
    columns {{
      name = "{tt}"
      type = "string"
    }}
    """)


    columns {
      name = "ticker"
      type = "string"
    }
    

    columns {
      name = "date_time"
      type = "string"
    }
    

    columns {
      name = "open"
      type = "string"
    }
    

    columns {
      name = "high"
      type = "string"
    }
    

    columns {
      name = "low"
      type = "string"
    }
    

    columns {
      name = "close"
      type = "string"
    }
    

    columns {
      name = "volume_fsym"
      type = "string"
    }
    

    columns {
      name = "volume_tsym"
      type = "string"
    }
    

    columns {
      name = "currency"
      type = "string"
    }
    

    columns {
      name = "delta"
      type = "string"
    }
    

    columns {
      name = "time_stamp"
      type = "string"
    }
    

    columns {
      name = "year"
      type = "string"
    }
    

    columns {
      name = "month"
      type = "string"
    }
    

    columns {
      name = "day"
      type = "string"
    }
    

    columns {
 